# MBA obfuscation

In [1]:
# Original expression
# ---
# (x | y) + (x ^ y)

# MBA rewrite rules
# ---
# x | y -> x + (~x & y)
# x ^ y -> x + y - 2*(x & y)
# x + y -> (x | y) + (~x | y) - ~x

In [2]:
"""
Prove the semantic equivalence of the operators
and their rewritten (more complex) form.
In other words, prove the correctness of the
MBA rewrite rules.
"""
from z3 import *
x, y = BitVecs("x y", 64)

# OR
prove(x | y == x + (~x & y))

# XOR
prove(x ^ y == x + y - 2*(x & y))

# ADD
prove(x + y == (x | y) + (~x | y) - ~x)

proved
proved
proved


In [3]:
"""
Replace 'x' and 'y' by 'L' and 'R' in rewrite expressions.

This eases the task of rewriting later, and helps visualize
the fact that we can apply rewrite rules not only to single
variables, but also to whole (Left and Right) (sub)expressions.
"""
mba_rewrite_or = "x + (~x & y)".replace('x', 'L').replace('y', 'R')
mba_rewrite_xor = "x + y - 2*(x & y)".replace('x', 'L').replace('y', 'R')
mba_rewrite_add = "(x | y) + (~x | y) - ~x".replace('x', 'L').replace('y', 'R')

print(f"MBA REWRITE OR\n---\n{mba_rewrite_or}\n")
print(f"MBA REWRITE XOR\n---\n{mba_rewrite_xor}\n")
print(f"MBA REWRITE ADD\n---\n{mba_rewrite_add}\n")

MBA REWRITE OR
---
L + (~L & R)

MBA REWRITE XOR
---
L + R - 2*(L & R)

MBA REWRITE ADD
---
(L | R) + (~L | R) - ~L



In [4]:
"""
Generate AST representation of the non-obfuscated (simple)
expression we will deal with.
"""

import ast
import astunparse

simp = "(x | y) + (x ^ y)"
expr = ast.parse(simp, mode="eval")

print(ast.dump(expr, indent=4))

Expression(
    body=BinOp(
        left=BinOp(
            left=Name(id='x', ctx=Load()),
            op=BitOr(),
            right=Name(id='y', ctx=Load())),
        op=Add(),
        right=BinOp(
            left=Name(id='x', ctx=Load()),
            op=BitXor(),
            right=Name(id='y', ctx=Load()))))


In [5]:
"""
Extract binary operations (i.e. operations with two operands)
from the AST representation of the expression, from bottom to top.
"""
def bottomUpBFS(node):
    q = []
    q.append(node)
    
    bin_ops = []
    
    while q:
        cur = q.pop(0)
        
        if hasattr(cur, 'left'):
            q.append(cur.left)
            
        if hasattr(cur, 'right'):
            q.append(cur.right)
            
        if isinstance(cur, ast.BinOp):
            bin_ops.append(cur)
            
    return bin_ops

In [6]:
"""
Apply MBA rewrite rules to obfuscate an expression.

Rewrite rules are applied to the whole (sub)expressions
that conform Left and Right operands for the
binary operations covered (OR, XOR and ADD).
"""
def rewriteMBA(root):
    bin_ops = bottomUpBFS(root)
    
    while bin_ops:
        cur = bin_ops.pop()
        new = None
        
        L = astunparse.unparse(cur.left)[:-1]
        R = astunparse.unparse(cur.right)[:-1]
        
        if isinstance(cur.op, ast.BitOr):
            # print ("OR rewrite")
            new = ast.parse(mba_rewrite_or.replace('L', L).replace('R', R), mode="eval").body
            
        if isinstance(cur.op, ast.BitXor):
            # print ("XOR rewrite")
            new = ast.parse(mba_rewrite_xor.replace('L', L).replace('R', R), mode="eval").body
            
        if isinstance(cur.op, ast.Add):
            # print ("ADD rewrite")
            new = ast.parse(mba_rewrite_add.replace('L', L).replace('R', R), mode="eval").body
            
        if new:
            cur.op = new.op
            cur.left = new.left
            cur.right = new.right
            
        # print(astunparse.unparse(expr))

In [7]:
"""
Apply rewrite rules (1st iteration)
"""
rewriteMBA(expr.body)
print(astunparse.unparse(expr))

((((x + ((~ x) & y)) | ((x + y) - (2 * (x & y)))) + ((~ (x + ((~ x) & y))) | ((x + y) - (2 * (x & y))))) - (~ (x + ((~ x) & y))))



In [8]:
"""
Prove the semantic equivalence of the original expression
and the obfuscated one derived from rewrite process
"""
x, y = BitVecs("x y", 64)

simp_z3 = (x | y) + (x ^ y)
obf2_z3 = ((((x + ((~ x) & y)) | ((x + y) - (2 * (x & y)))) + ((~ (x + ((~ x) & y))) | ((x + y) - (2 * (x & y))))) - (~ (x + ((~ x) & y))))

# Takes a few seconds
prove(simp_z3 == obf2_z3)

proved


In [9]:
"""
Apply rewrite rules (2nd iteration)
"""
rewriteMBA(expr.body)
print(astunparse.unparse(expr))

((((((((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x)) + ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((x | y) + ((~ x) | y)) - (~ x)) - (2 * (x & y))))) | ((~ (x + ((~ x) & y))) + ((~ (~ (x + ((~ x) & y)))) & ((((x | y) + ((~ x) | y)) - (~ x)) - (2 * (x & y)))))) + ((~ ((((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x)) + ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((x | y) + ((~ x) | y)) - (~ x)) - (2 * (x & y)))))) | ((~ (x + ((~ x) & y))) + ((~ (~ (x + ((~ x) & y)))) & ((((x | y) + ((~ x) | y)) - (~ x)) - (2 * (x & y))))))) - (~ ((((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x)) + ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((x | y) + ((~ x) | y)) - (~ x)) - (2 * (x & y))))))) - (~ (x + ((~ x) & y))))



In [10]:
x, y = BitVecs("x y", 64)

simp_z3 = (x | y) + (x ^ y)
obf3_z3 = ((((((((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x)) + ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((x | y) + ((~ x) | y)) - (~ x)) - (2 * (x & y))))) | ((~ (x + ((~ x) & y))) + ((~ (~ (x + ((~ x) & y)))) & ((((x | y) + ((~ x) | y)) - (~ x)) - (2 * (x & y)))))) + ((~ ((((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x)) + ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((x | y) + ((~ x) | y)) - (~ x)) - (2 * (x & y)))))) | ((~ (x + ((~ x) & y))) + ((~ (~ (x + ((~ x) & y)))) & ((((x | y) + ((~ x) | y)) - (~ x)) - (2 * (x & y))))))) - (~ ((((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x)) + ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((x | y) + ((~ x) | y)) - (~ x)) - (2 * (x & y))))))) - (~ (x + ((~ x) & y))))

# Takes quite some minutes
# prove(simp_z3 == obf3_z3)

In [11]:
"""
Apply rewrite rules (3rd iteration)
"""
rewriteMBA(expr.body)
print(astunparse.unparse(expr))

((((((((((((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x)) | ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y))))) + ((~ (((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x))) | ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y)))))) - (~ (((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (

In [12]:
x, y = BitVecs("x y", 64)

simp_z3 = (x | y) + (x ^ y)
obf4_z3 = ((((((((((((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x)) | ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y))))) + ((~ (((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x))) | ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y)))))) - (~ (((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x)))) + ((~ ((((((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x)) | ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y))))) + ((~ (((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x))) | ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y)))))) - (~ (((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x))))) & ((((~ (x + ((~ x) & y))) | ((~ (~ (x + ((~ x) & y)))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y))))) + ((~ (~ (x + ((~ x) & y)))) | ((~ (~ (x + ((~ x) & y)))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y)))))) - (~ (~ (x + ((~ x) & y))))))) | ((~ ((((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x)) + ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((x | y) + ((~ x) | y)) - (~ x)) - (2 * (x & y)))))) + ((~ (~ ((((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x)) + ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((x | y) + ((~ x) | y)) - (~ x)) - (2 * (x & y))))))) & ((((~ (x + ((~ x) & y))) | ((~ (~ (x + ((~ x) & y)))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y))))) + ((~ (~ (x + ((~ x) & y)))) | ((~ (~ (x + ((~ x) & y)))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y)))))) - (~ (~ (x + ((~ x) & y)))))))) + ((~ (((((((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x)) | ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y))))) + ((~ (((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x))) | ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y)))))) - (~ (((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x)))) + ((~ ((((((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x)) | ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y))))) + ((~ (((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x))) | ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y)))))) - (~ (((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x))))) & ((((~ (x + ((~ x) & y))) | ((~ (~ (x + ((~ x) & y)))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y))))) + ((~ (~ (x + ((~ x) & y)))) | ((~ (~ (x + ((~ x) & y)))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y)))))) - (~ (~ (x + ((~ x) & y)))))))) | ((~ ((((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x)) + ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((x | y) + ((~ x) | y)) - (~ x)) - (2 * (x & y)))))) + ((~ (~ ((((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x)) + ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((x | y) + ((~ x) | y)) - (~ x)) - (2 * (x & y))))))) & ((((~ (x + ((~ x) & y))) | ((~ (~ (x + ((~ x) & y)))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y))))) + ((~ (~ (x + ((~ x) & y)))) | ((~ (~ (x + ((~ x) & y)))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y)))))) - (~ (~ (x + ((~ x) & y))))))))) - (~ (((((((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x)) | ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y))))) + ((~ (((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x))) | ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y)))))) - (~ (((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x)))) + ((~ ((((((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x)) | ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y))))) + ((~ (((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x))) | ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y)))))) - (~ (((((x + ((~ x) & ((~ x) & y))) | ((~ x) + ((~ (~ x)) & ((~ x) & y)))) + ((~ (x + ((~ x) & ((~ x) & y)))) | ((~ x) + ((~ (~ x)) & ((~ x) & y))))) - (~ (x + ((~ x) & ((~ x) & y))))) - (~ x))))) & ((((~ (x + ((~ x) & y))) | ((~ (~ (x + ((~ x) & y)))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y))))) + ((~ (~ (x + ((~ x) & y)))) | ((~ (~ (x + ((~ x) & y)))) & ((((((x + ((~ x) & y)) | ((~ x) + ((~ (~ x)) & y))) + ((~ (x + ((~ x) & y))) | ((~ x) + ((~ (~ x)) & y)))) - (~ (x + ((~ x) & y)))) - (~ x)) - (2 * (x & y)))))) - (~ (~ (x + ((~ x) & y))))))))) - (~ ((((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x)) + ((~ (((x | ((~ x) & y)) + ((~ x) | ((~ x) & y))) - (~ x))) & ((((x | y) + ((~ x) | y)) - (~ x)) - (2 * (x & y))))))) - (~ (x + ((~ x) & y))))

# I don't think it will ever terminate :S
# prove(simp_z3 == obf3_z3)

# Symbolic execution

In [13]:
"""
Adapted from ./../msynth/scripts/symbolic_simplification.py

Extract the symbolic expression for RAX register in a given
basic block at start_addr.
"""
from miasm.analysis.binary import Container
from miasm.analysis.machine import Machine
from miasm.core.locationdb import LocationDB
from miasm.ir.symbexec import SymbolicExecutionEngine

def getRaxExpr(file_path, start_addr):
    # symbol table
    loc_db = LocationDB()

    # open the binary for analysis
    container = Container.from_stream(open(file_path, 'rb'), loc_db)

    # cpu abstraction
    machine = Machine(container.arch)

    # init disassemble engine
    mdis = machine.dis_engine(container.bin_stream, loc_db=loc_db)

    # initialize intermediate representation
    lifter = machine.lifter_model_call(mdis.loc_db)

    # disassemble the function at address
    asm_block = mdis.dis_block(start_addr)

    # lift to Miasm IR
    ira_cfg = lifter.new_ircfg()
    lifter.add_asmblock_to_ircfg(asm_block, ira_cfg)

    # init symbolic execution engine
    sb = SymbolicExecutionEngine(lifter)

    # symbolically execute basic block
    sb.run_block_at(ira_cfg, start_addr)
    
    # return the expression of rax (return value)
    return sb.eval_exprid(lifter.arch.regs.RAX)

In [14]:
"""
Extract symbolic expression for RAX, with respect to the
input variables (registers RDI and RSI) of the scramble
function in scramble1
"""
file_path = "./scramble1"
addr = 0x1149

rax_scramble1 = getRaxExpr(file_path, addr)
print("RAX from scramble1\n---")
print(rax_scramble1)

RAX from scramble1
---
(RDI ^ RSI) + (RDI | RSI)


In [15]:
"""
Extract symbolic expression for RAX, with respect to the
input variables (registers RDI and RSI) of the scramble
function in scramble2
"""
file_path = "./scramble2"
addr = 0x1149

rax_scramble2 = getRaxExpr(file_path, addr)
print("RAX from scramble2\n---")
print(rax_scramble2)

RAX from scramble2
---
RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + ((RDI + RSI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE) | (RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)))) + ((RDI + RSI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE) | ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF)) + 0x1


In [16]:
"""
Extract symbolic expression for RAX, with respect to the
input variables (registers RDI and RSI) of the scramble
function in scramble3
"""
file_path = "./scramble3"
addr = 0x1149

rax_scramble3 = getRaxExpr(file_path, addr)
print("RAX from scramble3\n---")
print(rax_scramble3)

RAX from scramble3
---
((RDI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + (RDI | RSI) + (RSI | (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0x1) & ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF)) + RDI * 0x2 + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) * 0x2 + ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + ((RDI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + (RDI | RSI) + (RSI | (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0x1) & ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF))) | (((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) & (RDI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + (RDI | RSI) + (RSI | (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0x1)) + -RDI + -(RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0xFFFFFFFFFFFFFFFF)) + ((((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) & (RDI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + (RDI | RSI) + (RSI | (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0x1)) + -RDI + -(RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0xFFFFFFFFFFFFFFFF) | ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + ((RDI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + (RDI | RSI) + (R

In [17]:
"""
Extract symbolic expression for RAX, with respect to the
input variables (registers RDI and RSI) of the scramble
function in scramble4
"""
file_path = "./scramble4"
addr = 0x1149

rax_scramble4 = getRaxExpr(file_path, addr)
print("RAX from scramble4\n---")
print(rax_scramble4)

RAX from scramble4
---
((RDI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + (RDI | RSI) + (RSI | (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0x1) & ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF)) + ((RDI * 0xFFFFFFFFFFFFFFFE + -(RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + -((((RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + RDI * 0x2 + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) | ((RDI & RSI) + -RDI + 0xFFFFFFFFFFFFFFFF)) + (((RDI & RSI) + -RDI + 0xFFFFFFFFFFFFFFFF) | ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF)) + 0x2) & ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF)) | ((RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + RDI * 0x2 + ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) | ((RDI & RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + -RDI + 0xFFFFFFFFFFFFFFFF)) + (((RDI & RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + -RDI + 0xFFFFFFFFFFFFFFFF) | ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF)) + 0x2)) + -((((RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + RDI * 0x2

In [18]:
"""
Load miasm translator to convert miasm expressions into
z3 expressions
"""
from miasm.ir.translators.z3_ir import TranslatorZ3
translator = TranslatorZ3()

In [19]:
"""
Compare RAX expression from symbolic execution with
simplified expression through basic z3's SMT simplification
of the scramble function in scramble1
"""
print(f"Original RAX scramble1:\n---\n{rax_scramble1}\n")

rax_scramble1_z3 = translator.from_expr(rax_scramble1)
print(f"SMT simplified RAX scramble1:\n---\n{simplify(rax_scramble1_z3)}\n")

Original RAX scramble1:
---
(RDI ^ RSI) + (RDI | RSI)

SMT simplified RAX scramble1:
---
(RDI ^ RSI) + (RDI | RSI)



In [20]:
"""
Compare RAX expression from symbolic execution with
simplified expression through basic z3's SMT simplification
of the scramble function in scramble2
"""
print(f"Original RAX scramble2:\n---\n{rax_scramble2}\n")

rax_scramble2_z3 = translator.from_expr(rax_scramble2)
print(f"SMT simplified RAX scramble2:\n---\n{simplify(rax_scramble2_z3)}\n")

Original RAX scramble2:
---
RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + ((RDI + RSI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE) | (RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)))) + ((RDI + RSI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE) | ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF)) + 0x1

SMT simplified RAX scramble2:
---
1 +
RDI +
~(~RSI | RDI) +
(RDI + RSI + 18446744073709551614*~(~RDI | ~RSI) |
 RDI + ~(~RSI | RDI)) +
(RDI + RSI + 18446744073709551614*~(~RDI | ~RSI) |
 ~(RDI + ~(~RSI | RDI)))



In [21]:
"""
Compare RAX expression from symbolic execution with
simplified expression through basic z3's SMT simplification
of the scramble function in scramble3
"""
print(f"Original RAX scramble3:\n---\n{rax_scramble3}\n")

rax_scramble3_z3 = translator.from_expr(rax_scramble3)
print(f"SMT simplified RAX scramble3:\n---\n{simplify(rax_scramble3_z3)}\n")

Original RAX scramble3:
---
((RDI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + (RDI | RSI) + (RSI | (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0x1) & ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF)) + RDI * 0x2 + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) * 0x2 + ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + ((RDI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + (RDI | RSI) + (RSI | (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0x1) & ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF))) | (((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) & (RDI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + (RDI | RSI) + (RSI | (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0x1)) + -RDI + -(RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0xFFFFFFFFFFFFFFFF)) + ((((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) & (RDI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + (RDI | RSI) + (RSI | (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0x1)) + -RDI + -(RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0xFFFFFFFFFFFFFFFF) | ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + ((RDI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + (RDI | RSI)

In [22]:
"""
Compare RAX expression from symbolic execution with
simplified expression through basic z3's SMT simplification
of the scramble function in scramble4
"""
print(f"Original RAX scramble4:\n---\n{rax_scramble4}\n")

rax_scramble4_z3 = translator.from_expr(rax_scramble4)
print(f"SMT simplified RAX scramble4:\n---\n{simplify(rax_scramble4_z3)}\n")

Original RAX scramble4:
---
((RDI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + (RDI | RSI) + (RSI | (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0x1) & ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF)) + ((RDI * 0xFFFFFFFFFFFFFFFE + -(RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + -((((RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + RDI * 0x2 + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) | ((RDI & RSI) + -RDI + 0xFFFFFFFFFFFFFFFF)) + (((RDI & RSI) + -RDI + 0xFFFFFFFFFFFFFFFF) | ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF)) + 0x2) & ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF)) | ((RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + RDI * 0x2 + ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) | ((RDI & RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + -RDI + 0xFFFFFFFFFFFFFFFF)) + (((RDI & RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + -RDI + 0xFFFFFFFFFFFFFFFF) | ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF)) + 0x2)) + -((((RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + RDI 

# Program synthesis

In [23]:
"""
Load synthesis Simplifier from msynth tool and
initialize it with given oracle database
"""
from msynth import Simplifier
oracle_path = "./../msynth/oracle.pickle"

simplifier = Simplifier(oracle_path)

In [24]:
"""
Compare RAX expression from symbolic execution with
synthesized expression through msynth synthesis
simplification of the scramble function in scramble1
"""
rax_scramble1_synth = simplifier.simplify(rax_scramble1)

print(f"RAX SYMEX scramble1:\n---\n{rax_scramble1}\n")
print(f"RAX SYNTH scramble1:\n---\n{rax_scramble1_synth}\n")

RAX SYMEX scramble1:
---
(RDI ^ RSI) + (RDI | RSI)

RAX SYNTH scramble1:
---
(RDI ^ RSI) + (RDI | RSI)



In [25]:
"""
Compare RAX expression from symbolic execution with
synthesized expression through msynth synthesis
simplification of the scramble function in scramble2
"""
rax_scramble2_synth = simplifier.simplify(rax_scramble2)

print(f"RAX SYMEX scramble2:\n---\n{rax_scramble2}\n")
print(f"RAX SYNTH scramble2:\n---\n{rax_scramble2_synth}\n")

RAX SYMEX scramble2:
---
RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + ((RDI + RSI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE) | (RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)))) + ((RDI + RSI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE) | ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF)) + 0x1

RAX SYNTH scramble2:
---
(RDI ^ RSI) + (RDI | RSI)



In [26]:
"""
Compare RAX expression from symbolic execution with
synthesized expression through msynth synthesis
simplification of the scramble function in scramble3
"""
rax_scramble3_synth = simplifier.simplify(rax_scramble3)

print(f"RAX SYMEX scramble3:\n---\n{rax_scramble3}\n")
print(f"RAX SYNTH scramble3:\n---\n{rax_scramble3_synth}\n")

RAX SYMEX scramble3:
---
((RDI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + (RDI | RSI) + (RSI | (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0x1) & ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF)) + RDI * 0x2 + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) * 0x2 + ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + ((RDI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + (RDI | RSI) + (RSI | (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0x1) & ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF))) | (((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) & (RDI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + (RDI | RSI) + (RSI | (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0x1)) + -RDI + -(RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0xFFFFFFFFFFFFFFFF)) + ((((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) & (RDI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + (RDI | RSI) + (RSI | (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0x1)) + -RDI + -(RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0xFFFFFFFFFFFFFFFF) | ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + ((RDI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + (RDI | RSI) + 

In [27]:
"""
Compare RAX expression from symbolic execution with
synthesized expression through msynth synthesis
simplification of the scramble function in scramble4
"""
rax_scramble4_synth = simplifier.simplify(rax_scramble4)

print(f"RAX SYMEX scramble4:\n---\n{rax_scramble4}\n")
print(f"RAX SYNTH scramble4:\n---\n{rax_scramble4_synth}\n")

RAX SYMEX scramble4:
---
((RDI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + (RDI | RSI) + (RSI | (RDI ^ 0xFFFFFFFFFFFFFFFF)) + 0x1) & ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF)) + ((RDI * 0xFFFFFFFFFFFFFFFE + -(RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + -((((RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + RDI * 0x2 + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) | ((RDI & RSI) + -RDI + 0xFFFFFFFFFFFFFFFF)) + (((RDI & RSI) + -RDI + 0xFFFFFFFFFFFFFFFF) | ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF)) + 0x2) & ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF)) | ((RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + RDI * 0x2 + ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) | ((RDI & RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + -RDI + 0xFFFFFFFFFFFFFFFF)) + (((RDI & RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + -RDI + 0xFFFFFFFFFFFFFFFF) | ((RDI + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) ^ 0xFFFFFFFFFFFFFFFF)) + 0x2)) + -((((RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + RDI * 0